In [76]:
import folium
import branca.colormap as cm
import numpy as np
import pandas as pd

def generate_aadt_map(csv_file, num_subsegments=10):
    """
    Generate a scatter map of AADT (Annual Average Daily Traffic) with colors varying by AADT value based on a CSV file.
    
    Parameters:
    - csv_file (str): Path to the CSV file, which must contain the columns "Latitude", "Longitude", and "VEHICLE_AADT_TOTAL".
    
    Returns:
    - folium.Map object
    """
    # 1. Read the CSV file
    df = pd.read_csv(csv_file)
    df["Longitude"] = -df["Longitude"] 
    filtered_df =  df[(df["CNTY"].isin(["SD", "LA", "ORA"])) & (df["Year"] == 2023)].copy()
    filtered_df["POSTMILE_DIFF"] = filtered_df["POSTMILE"].diff()
    filtered_df = filtered_df[filtered_df["POSTMILE_DIFF"] < 2]  

    
    # 2. Create a Folium map (set the center to the mean of coordinates)
    m = folium.Map(
        location=[filtered_df["Latitude"].mean(), filtered_df["Longitude"].mean()], 
        zoom_start=7, 
        tiles="OpenStreetMap"
    )

    # 3. Create a color map (for the scatter plot with a gradient of AADT values)
    colormap = cm.LinearColormap(
        colors=["blue", "green", "yellow", "red"],  
        vmin=filtered_df["AADT"].min(),  
        vmax=filtered_df["AADT"].max()
    )

    # 4. Loop through the data and plot the scatter points
    # 4绘制平滑渐变线条
    for i in range(len(filtered_df) - 1):
        lat1, lon1, aadt1 = filtered_df.iloc[i][["Latitude", "Longitude", "AADT"]]
        lat2, lon2, aadt2 = filtered_df.iloc[i + 1][["Latitude", "Longitude", "AADT"]]

        # 生成经纬度插值（细分线段）
        lats = np.linspace(lat1, lat2, num_subsegments)
        lons = np.linspace(lon1, lon2, num_subsegments)
        aadts = np.linspace(aadt1, aadt2, num_subsegments)  # AADT 也插值，使颜色渐变

        # 画小线段（颜色渐变）
        for j in range(num_subsegments - 1):
            color = colormap(aadts[j])  # 计算当前小线段的颜色

            folium.PolyLine(
                locations=[(lats[j], lons[j]), (lats[j + 1], lons[j + 1])],
                color=color,
                weight=5,
                opacity=0.8
            ).add_to(m)

    # 5添加 AADT 颜色图例
    colormap.caption = "predicted AADT Traffic Data in 2023"
    colormap.add_to(m)

    # 6返回地图对象
    return m


In [77]:
aadt_map = generate_aadt_map("D:/143/project/predicted_AADT_2023-2025.csv")  
display(aadt_map)